In [0]:
table_name = "workspace.default.credit_risk_highacc"  # change if needed

df = spark.table(table_name)
df.printSchema()
display(df.limit(10))
print("Rows:", df.count())


root
 |-- customer_id: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- loan_balance: long (nullable = true)
 |-- current_income: long (nullable = true)
 |-- risk_score: double (nullable = true)
 |-- high_risk: long (nullable = true)



customer_id customer_name loan_balance current_income risk_score high_risk CUST_1 Kavya Pandey 768761 148148 0.531 0 CUST_2 Riya Naidu 280784 77341 0.398 0 CUST_3 Aditya Joshi 241324 39085 0.636 1 CUST_4 Sneha Chopra 494273 13750 1.0 1 CUST_5 Saanvi Singh 695409 43903 1.0 1 CUST_6 Diya Mehta 408834 211561 0.196 0 CUST_7 Nikhil Menon 243612 99036 0.273 0 CUST_8 Rohit Joshi 721413 95954 0.78 1 CUST_9 Aditya Reddy 929424 125305 0.779 1 CUST_10 Diya Mehta 316081 208753 0.158 0

Rows: 500


In [0]:
from pyspark.sql.functions import col

data = (df
    .withColumn("loan_balance",  col("loan_balance").cast("double"))
    .withColumn("current_income",col("current_income").cast("double"))
    .withColumn("risk_score",    col("risk_score").cast("double"))
    .withColumn("label",         col("high_risk").cast("int"))
    .withColumn("dti",           (col("loan_balance")/(col("current_income")+1e-6)))  # Debt-to-Income
    .drop("customer_id","customer_name","high_risk")
)

display(data.limit(5))
display(data.groupBy("label").count())


loan_balance current_income risk_score label dti 768761.0 148148.0 0.531 0 5.1891419391069125 280784.0 77341.0 0.398 0 3.6304676691065483 241324.0 39085.0 0.636 1 6.174337981164786 494273.0 13750.0 1.0 1 35.94712727011294 695409.0 43903.0 1.0 1 15.839669270531862

label count 0 371 1 129

In [0]:
train_df, test_df = data.randomSplit([0.8, 0.2], seed=42)
print("Train:", train_df.count(), "Test:", test_df.count())


Train: 408 Test: 92


In [0]:
from pyspark.ml.feature import VectorAssembler

feature_cols = ["loan_balance","current_income","risk_score","dti"]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

train_vec = assembler.transform(train_df).select("features","label")
test_vec  = assembler.transform(test_df).select("features","label")


In [0]:
from pyspark.ml.classification import LogisticRegression, GBTClassifier

# Logistic Regression (fast, interpretable)
lr = LogisticRegression(featuresCol="features", labelCol="label", maxIter=100, regParam=0.02)
lr_model = lr.fit(train_vec)
pred_lr = lr_model.transform(test_vec)

# Gradient-Boosted Trees (often best for tabular risk)
gbt = GBTClassifier(featuresCol="features", labelCol="label", maxDepth=5, maxIter=80, stepSize=0.1)
gbt_model = gbt.fit(train_vec)   # (uses features/label only)
pred_gbt = gbt_model.transform(test_vec)


In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

def eval_bin(pred):
    acc = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy").evaluate(pred)
    f1  = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1").evaluate(pred)
    auc = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderROC").evaluate(pred)
    aupr= BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderPR").evaluate(pred)
    return acc,f1,auc,aupr

print("LR  :", eval_bin(pred_lr))
print("GBT :", eval_bin(pred_gbt))


LR  : (0.967391304347826, 0.9665403525424734, 1.0, 1.0)
GBT : (1.0, 1.0, 1.0, 1.0)


In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType, IntegerType

chosen = pred_gbt  # or pred_lr if you prefer
# probability is a 2-element vector: [prob_class0, prob_class1]
to_prob1 = udf(lambda v: float(v[1]), DoubleType())
scored = chosen.withColumn("prob_risk", to_prob1("probability"))

# pick a threshold (e.g., 0.4 to catch more high-risk)
thr = 0.40
from pyspark.sql.functions import when, col
scored = scored.withColumn("pred_at_thr", when(col("prob_risk") >= thr, 1).otherwise(0))

display(scored.select("label","prob_risk","pred_at_thr").limit(20))


label prob_risk pred_at_thr 0 0.014242545558209363 0 0 0.014242545558209363 0 0 0.014242545558209363 0 0 0.014242545558209363 0 0 0.014242545558209363 0 0 0.014242545558209363 0 0 0.014242545558209363 0 0 0.014242545558209363 0 0 0.014242545558209363 0 0 0.014242545558209363 0 0 0.014242545558209363 0 0 0.014242545558209363 0 0 0.014242545558209363 0 0 0.014242545558209363 0 0 0.014242545558209363 0 0 0.014242545558209363 0 0 0.014242545558209363 0 0 0.014242545558209363 0 0 0.014242545558209363 0 0 0.014242545558209363 0

In [0]:
pdf = scored.select("label","pred_at_thr").toPandas()

import pandas as pd
cm = pd.crosstab(pdf["label"], pdf["pred_at_thr"], rownames=["Actual"], colnames=["Predicted"], margins=True)
cm


Predicted   0   1  All
Actual                
0          70   0   70
1           0  22   22
All        70  22   92

In [0]:
from pyspark.sql import Row

# Keep whichever model you chose:
final_model = gbt_model   # or lr_model

def score_new(app_rows):
    """
    app_rows: list of dicts with keys: loan_balance, current_income, risk_score
    Returns a Spark DataFrame with prob_risk and risk_flag.
    """
    # create df
    new_df = spark.createDataFrame([Row(**r) for r in app_rows])
    # add dti
    new_df = new_df.withColumn("loan_balance", col("loan_balance").cast("double")) \
                   .withColumn("current_income", col("current_income").cast("double")) \
                   .withColumn("risk_score", col("risk_score").cast("double")) \
                   .withColumn("dti", (col("loan_balance")/(col("current_income")+1e-6)))

    # assemble
    feats = assembler.transform(new_df).select("loan_balance","current_income","risk_score","dti","features")
    # predict
    out = final_model.transform(feats)
    out = out.withColumn("prob_risk", to_prob1("probability")) \
             .withColumn("risk_flag", when(col("prob_risk") >= thr, 1).otherwise(0))
    return out.select("loan_balance","current_income","risk_score","dti","prob_risk","risk_flag")

# Example “incoming” mini-batch
incoming = [
    {"loan_balance": 450000, "current_income": 35000,  "risk_score": 0.78},
    {"loan_balance": 120000, "current_income": 120000, "risk_score": 0.22},
    {"loan_balance": 800000, "current_income": 60000,  "risk_score": 0.65},
]
display(score_new(incoming))


loan_balance current_income risk_score dti prob_risk risk_flag 450000.0 35000.0 0.78 12.85714285677551 0.9857574544417907 1 120000.0 120000.0 0.22 0.9999999999916668 0.014242545558209363 0 800000.0 60000.0 0.65 13.333333333111112 0.9857574544417907 1